## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
# import matplotlib.image as Image
import os
import albumentations as A
import sys
import random
import cv2
import gc

import tensorflow as tf
from tensorflow import keras

In [ ]:
direcory_train = '../input/petfinder-pawpularity-score/train/'
directory_test = '../input/petfinder-pawpularity-score/test/'

csv_train = '../input/petfinder-pawpularity-score/train.csv'
csv_test = '../input/petfinder-pawpularity-score/test.csv'

In [ ]:
data = pd.read_csv(csv_train)
data.head()

## EDA

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
fig = plt.figure(figsize=(20,5))
arr, bins, patches = plt.hist(data.Pawpularity, bins = 199)
plt.xticks(range(1,101), rotation=90)
plt.show()

### Manipulate dataset labels
Manipulating dataset labels is necessary. Because 99, 98, 97 and 100 scored images are very similar. And also other images. So it's very hard for the model to predict accurately. Trying to manipulate the images, and trying out the accuracy, in case it helps!

In [ ]:
dataPawpularity = np.array(data.Pawpularity, dtype=np.float32)

# Frequency for the first half is huge. So we used more slicing than the later half
for i in range(5, 51, 5):
    dataPawpularity[(dataPawpularity>(i-5)) & (dataPawpularity<=i)] = (2*i - 5)/2.0

for i in range(55, 101, 5):
    dataPawpularity[(dataPawpularity>(i-5)) & (dataPawpularity<=i)] = i
    
print(np.unique(dataPawpularity))

In [ ]:
# Let's check if it still follows the trend

fig = plt.figure(figsize=(20,5))
arr, bins, patches = plt.hist(dataPawpularity, bins = 41)
plt.xticks(range(1,101), rotation=90)
plt.show()

In [ ]:
data.Pawpularity = dataPawpularity
del dataPawpularity

### View some images with low and high pawpularity

In [ ]:
pawpularity_0 = data[data.Pawpularity <= 5]
pawpularity_30 = data[(data.Pawpularity >= 25) & (data.Pawpularity <= 35)]
pawpularity_100 = data[data.Pawpularity >= 95]

In [ ]:
def show_images(paws):
    fig = plt.figure(figsize=(10,10), constrained_layout=True)
    grids = fig.add_gridspec(3,3)
    
    for i in range(3):
        for j in range(3):
            img = Image.open(direcory_train + paws.Id.iloc[i*3 + j] + '.jpg')
            ax = fig.add_subplot(grids[i,j])
            ax.imshow(img)
    
    plt.show()

In [ ]:
print('Pawpularity <= 5')
show_images(pawpularity_0)

In [ ]:
print('Pawpularity >= 25 and <=35')
show_images(pawpularity_30)

In [ ]:
print('Pawpularity >= 95')
show_images(pawpularity_100)

## Convert images to nparray

In [ ]:
data_np = []
target = data.Pawpularity

img_shape = (224,224,3)

In [ ]:
def make_nparray(data, directory):
    im_array = np.zeros((data.shape[0], img_shape[0], img_shape[1], 3), dtype=np.uint8)
    
    for i in range(data.shape[0]):
        img = Image.open(directory + data.Id.iloc[i] + '.jpg')
        img = img.resize((img_shape[0], img_shape[1]))
        im_array[i] = np.array(img, dtype=np.uint8)
    
    return im_array

In [ ]:
# Run this on kaggle notebook
# data_np = make_nparray(data, direcory_train)

In [ ]:
# No need to run this on local machine. simply save and load from local machine!

def load_data():
    global data_np
    
    import pickle

    # data_file = open('data_file.pkl', 'wb')
    # pickle.dump(data_np, data_file)
    # data_file.close()

    data_file = open('../input/petfinder-numpy300300/data_file.pkl', 'rb')
    data_np = pickle.load(data_file)
    data_file.close()

load_data()

## Data augmentation

In [ ]:
class Paws:
    
    def __init__(self, data_np):
        assert type(data_np) == np.ndarray
        assert len(data_np.shape) == 4
        self.data = data_np.copy()
    
    def getAugmented(self, raw = 0, p = 0.3):
        global img_shape
        
        if raw == 0:
            return self.data
        else:
            print('Augmenting!')
            
            augmented = np.zeros((self.data.shape[0], img_shape[0], img_shape[1], img_shape[2]), dtype=np.uint8)
            cut = random.randint(img_shape[0]-30,img_shape[0])

            transformer = A.Compose([
                A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0, rotate_limit=0,p=p,border_mode=cv2.INTER_NEAREST),
                A.RandomCrop(cut,cut,p=p),
                A.HorizontalFlip(p=p),
                A.Rotate(limit=15, p=p),
#                 A.VerticalFlip(p=p),
                #     A.GridDropout(ratio=0.3,random_offset=True,holes_number_x=1, holes_number_y=1,always_apply=True),
                #     A.Cutout(num_holes=1, max_h_size=100, max_w_size=100,fill_value=0, always_apply=True ),
                A.CoarseDropout(max_holes=1, max_height=50, max_width=50,fill_value=0, p=0.3),
                A.Resize(height=img_shape[0],width=img_shape[1], always_apply=True)
            ])
            
            for i in range(self.data.shape[0]):
                transformed = transformer(image=data_np[i])
                augmented[i] = transformed['image']
            
        return augmented
    
    def getOriginalData(self):
        return self.data;

## Callbacks

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
start_lr = 0.000625
min_lr = 0.00001
max_lr = 0.001
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

current_epoch = 0

def lrfn(epoch):
    global current_epoch
#     print('current epoch:', current_epoch)
    
#     if current_epoch < rampup_epochs:
#         lr = (max_lr - start_lr)/rampup_epochs * current_epoch + start_lr
#         current_epoch += 1
#         return lr
#     elif current_epoch < rampup_epochs + sustain_epochs:
#         current_epoch += 1
#         return max_lr
#     else:
#         lr = (max_lr - min_lr) * exp_decay**(current_epoch-rampup_epochs-sustain_epochs) + min_lr
#         current_epoch += 1
#         return lr
    
    if epoch < rampup_epochs:
        lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        return lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rang = np.arange(50)
y = [lrfn(x) for x in rang]
plt.plot(rang, y)
print('Learning rate per epoch:')
current_epoch = 0

In [ ]:
file_it = 1

def getFilename():
    return 'it-'+str(file_it)+'vl-{val_loss:.2f}-ep-{epoch:02d}.hdf5'

cp_callback = keras.callbacks.ModelCheckpoint(filepath='best.hdf5',
                                              monitor='val_loss',
                                              verbose=1,
                                              mode='min', 
                                              save_weights_only=True, 
                                              save_best_only=True)

## Train

In [ ]:
# custom train

class Trainer:
    
    def __init__(self, model ,data, meta_data):
        if type(model) == str:
            self.model = self.loadModel(model)
        else:
            self.model = model
        self.meta_data = meta_data
        self.paws = Paws(data)
    
    def train(self,epochs:int, iterations:int, batch_size:int, validation_split:np.float32 = 0.15 ):
        global file_it
        file_it = 1
        
        for it in range(iterations):
            print('Iteration:{0}/{1}'.format(it+1,iterations))
            file_it = it + 1
            
            meta_model.fit(x=[self.paws.getAugmented(0), self.meta_data],
                           y=target, batch_size=batch_size,
                           epochs=epochs,
                           validation_split=validation_split,
                           callbacks=[lr_callback, es_callback, cp_callback ])
            gc.collect()
            
    def loadModel(self, file_name):
        self.model = keras.models.load_model(file_name)


In [ ]:
def make_model_with_metadata(lr):
    
    mobileNet = keras.applications.MobileNetV2(include_top=False,
                                               weights='../input/mobilenetv2weights/pre-trained.hdf5'
                                              )
    for l in mobileNet.layers:
        l.trainable = False
    
    meta_in = keras.layers.Input((12,))
    X = keras.layers.GlobalAveragePooling2D()(mobileNet.output)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Concatenate()([X, meta_in])
    X = keras.layers.Dense(100)(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Dense(1)(X)
    
    opt = keras.optimizers.Adam(lr)
    model = keras.models.Model(inputs = [mobileNet.input, meta_in], outputs = X)
    
    model.compile(optimizer=opt, loss='mean_squared_error', metrics= ['mean_squared_error'])
    
    return model

In [ ]:
meta_model = make_model_with_metadata(0.0001)

In [ ]:
# print(len(meta_model.layers))


In [ ]:
meta_data = data.loc[:, 'Subject Focus':'Blur']
meta_data_np = np.array(meta_data)
# meta_model.summary()

In [ ]:
# meta_model.fit(x=[data_np, meta_data_np],
#                y=target, batch_size=4,
#                epochs=10,
#                validation_split=0.15,
#                workers=6,
#                callbacks=[lr_callback, es_callback, cp_callback ])

trainer = Trainer(meta_model, data_np, meta_data_np)

In [ ]:
trainer.train(epochs=20, iterations= 3, batch_size=16)

## Test

In [ ]:
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_meta = test.loc[:, 'Subject Focus':]
test_meta = np.array(test_meta)

In [ ]:
test_np = make_nparray(test, directory_test)

In [ ]:
meta_model.load_weights('./best.hdf5')

### Output

In [ ]:
predictions = meta_model.predict([test_np, test_meta])

In [ ]:
submission = pd.DataFrame(predictions, columns=['Pawpularity'])
submission['Id'] = test.Id
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())